In [8]:
import tensorflow as tf
import numpy as np
from collections import deque
import random
import pandas as pd
import json

## Agent Environment

In [9]:
class AgentEnvironment:
    def __init__(self, environment, cost, num_states):
        self.environment = environment
        self.cost = cost
        self.num_states = num_states
        self.state = np.random.randint(0, self.num_states)

    def reset(self):
        # random action to cater for instances where some states were not visited by the agent
        n_actions = len(self.environment.keys())
        r_action = np.random.randint(0, n_actions)
        
        # obtain set of possible start states
        trans_mat = self.environment[r_action].index.tolist()

        # obtain the starting state
        self.state = np.random.choice(trans_mat)
        #self.state = np.random.randint(0, self.num_states)
        return self.state

    def step(self, state, action):
        # obtain the possible next states
        ## 
        #print('action {}'.format(action))
        #print('state {}'.format(state))
        
        pos_next_states = self.environment[action].columns.tolist()
        p_j = self.environment[action].loc[state].values
        next_state = np.random.choice(pos_next_states, p=p_j)
        cost = self.cost[action][state]
        done = False
        return next_state, cost, done

## Setting Up the Memory Externally for Future Import (Multiagent)

In [10]:
class AgentMemory:
    def __init__(self, max_length):
        self.memory = deque(maxlen=max_length)

    def add(self, experience):
        self.memory.append(experience)

    def sample(self, batch_size, num_states):
        batch_info = random.sample(self.memory, batch_size)
        states, actions, next_states, costs, dones = zip(*batch_info)
        #print(' memory next_states {}'.format(next_states)

        # onehot encode states and next states
        states = [int(s) for s in states]
        next_states = [int(ns) for ns in next_states]

        id_vector = np.eye(num_states)
        states = id_vector[states]
        next_states = id_vector[next_states]
        #actions
        return states, actions, next_states, costs, np.array(dones)
        

    def __len__(self):
        return len(self.memory)

## Building the Q Network
* Goal:
    - Considering that the solution to the Bellman is unique. Result consistency and reproducibility is important.

In [11]:
class AgentDQN(tf.keras.Model):
    def __init__(self, num_states, num_actions, hidden_dim):
        super(AgentDQN, self).__init__()
        # Define network layers
        self.h1 = tf.keras.layers.Dense(hidden_dim, activation='linear', kernel_initializer=tf.keras.initializers.GlorotNormal())
        self.h2 = tf.keras.layers.Dense(16, activation='linear', kernel_initializer=tf.keras.initializers.GlorotNormal())
        self.h3 = tf.keras.layers.Dense(8, activation = 'linear', kernel_initializer=tf.keras.initializers.GlorotNormal())
        self.q_vals = tf.keras.layers.Dense(num_actions, activation='linear')

    def call(self, state_onehot):
        # forward pass
        x = self.h1(state_onehot)
        x = self.h2(x)
        x = self.h3(x)
        q_values = self.q_vals(x)
        return q_values

## Compiling the DQN and Training Loop

In [12]:
class DQNBuild:
    def __init__(self, environment, cost, num_states, num_actions, hidden_dim, gamma = 0.95, lr=0.01, memory_length = 1000):
        # initialize the environment
        self.env = AgentEnvironment(environment=environment, cost=cost,num_states=num_states)
        # initialize the AgentDQN
        self.q_model = AgentDQN(num_states = num_states, num_actions = num_actions, hidden_dim=hidden_dim)
        # initalize the target network
        self.target_q_model = AgentDQN(num_states = num_states, num_actions = num_actions, hidden_dim=hidden_dim)
        # intialize the agent memory
        self.buffer = AgentMemory(max_length = memory_length)
        # discount rate
        self.gamma = gamma
        self.num_states = num_states
        self.num_actions = num_actions
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        self.train_step_counter = 0
        #self.target_update_period = target_update_period

    def update_target_network(self):
        # update target network at intervals to match the weights of the q-network
        self.target_q_model.set_weights(self.q_model.get_weights())

    def choose_action(self, state, epsilon):
        # choose action based on epsolon-greedy strategy with (np.argmin) for cost minimization
        if np.random.rand() < epsilon:
            return np.random.choice(self.num_actions)
        else:
            # onehot encode the state
            state_onehot = tf.expand_dims(tf.one_hot(state, self.num_states), axis=0)
            # predict q_values
            q_values = self.target_q_model(state_onehot)

            # select action with minimum q_value
            return tf.argmin(q_values, axis=1).numpy()[0] 


    def train_step(self, states, actions, costs, next_states, dones, batch_size, use_target_network, target_update_period):
        # obtain the loss function
        mse_ = tf.keras.losses.MeanSquaredError()

        with tf.GradientTape() as tape:

            # Calculate current Q-values
            current_q_values = self.q_model(states)
            #print(f'current q shape{current_q_values.shape}')
            
            # Get target Q-values base
            target_q_values = self.target_q_model(states).numpy() if use_target_network else  self.q_model(states).numpy()
            backup_q = self.target_q_model(next_states).numpy() if use_target_network else  self.q_model(next_states).numpy() # copy current q values
            
            # Calculate backup Q-values for next states
            #backup_q = self.q_model(next_states).numpy()
            #print(f'backup_q shape {backup_q.shape}')

            # Calculate target Q-values with Bellman equation
            target_q = costs + self.gamma * np.min(backup_q, axis=1) * (1-dones)
            target_q_values[np.arange(batch_size), actions]= target_q
            target_q_values = tf.convert_to_tensor(target_q_values)
            
            # Calculate loss
            loss = mse_(target_q_values, current_q_values)

        # Compute gradients
        grads = tape.gradient(loss, self.q_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.q_model.trainable_variables))

        # update the target network counted
        self.train_step_counter += 1
        if use_target_network and target_update_period and self.train_step_counter % target_update_period == 0:
            self.update_target_network()

        return loss

    def train(self, episodes, steps_per_episode, training_cost=1e5, batch_size=32, epsilon=1, epsilon_min=0.01, epsilon_decay=0.9995, episodic=False, terminal_period = False,
             use_target_network = False, target_update_period = None):
        # initialize old policy to be used for convergence check
        previous_policy = np.zeros(shape=(1, self.num_states))
        state = self.env.reset() # incase episodic is False

        training_cost = training_cost
        total_cost = 0
        
        
        for episode in range(episodes):
            terminal_period_counter = 0
            
            for step_ in range(steps_per_episode):
                terminal_period_counter += 1
                # choose action
                action = self.choose_action(state=state, epsilon=epsilon)

                #obtain the one step transition information
                next_state, cost, done = self.env.step(state, action)

                # update the dones if episodic
                if terminal_period and terminal_period_counter % terminal_period == 0:
                    done = True
                if terminal_period == False and episode == episodes-1 and step_ == steps_per_episode:
                    done = True

                # store experience in memory
                self.buffer.add((state, action, next_state, cost, done))

                if len(self.buffer)>= batch_size:
                    # unpack the batch_info
                    states, actions, next_states, costs, dones = self.buffer.sample(batch_size, num_states=self.num_states)

                    # train the model
                    loss = self.train_step(states = states, actions = actions, 
                                           next_states = next_states, costs = costs, 
                                           dones = dones, batch_size=batch_size, 
                                           use_target_network = use_target_network,
                                          target_update_period = target_update_period)


                # update state and total cost
                state = next_state
                training_cost -= cost
                total_cost += cost
                # epsilon decay
                epsilon = max(epsilon_min, epsilon*epsilon_decay)

            # check for convergence
            # obtain current policy
            current_policy = self.get_policy()
            if np.array_equal(current_policy, previous_policy):
                print(f'Convergence Reached With Statble Policy at {episode}')
                print(f'Optimal Policy: {current_policy}')
                break
            else:
                previous_policy = current_policy
                
            # reset the starting state if it is the case the the system resets every year
            if episodic:
                state = self.env.reset()
            

    def get_policy(self):
        # check if the current policy is equal to the new policy
        id_vector = np.eye(self.num_states)
        q_values = self.q_model(id_vector)
        return np.argmin(q_values, axis=1)

# Testing

In [13]:
# Load JSON data from the file
with open('transition_matrices/transition_matrices_3_agents.json', 'r') as file:
    json_string = file.read()      # Read JSON string from file
    agent_information = json.loads(json_string) # Parse the JSON string

## Preprocessing

In [14]:
transition_matrix, costs = agent_information

In [15]:
num_agents = len(costs)
num_actions = 2

# convert all
for agent in range(num_agents):
    transition_matrix[agent] = {int(k):v for k, v in transition_matrix[agent].items()}
    costs[agent] = {int(k) : v for k, v in costs[agent].items()}

    for action in range(num_actions):
        transition_matrix[agent][action] = pd.DataFrame(json.loads(transition_matrix[agent][action]))
        # obtain the columns
        agent_action_cols = [float(i) for i in transition_matrix[agent][action].columns.tolist()]
        transition_matrix[agent][action].columns = agent_action_cols
        costs[agent][action] = {int(k) : v for k, v in costs[agent][action].items()}
        

In [16]:
transition_matrix[1][1]

,0.0,1.0,2.0
0,0.0,0.5,0.5
1,1.0,0.0,0.0
2,0.0,1.0,0.0


In [17]:
transition_matrix[0][0].loc[2].values

array([0.66666667, 0.        , 0.33333333])

## obtain sample agent

In [18]:
transition_matrix_ = transition_matrix[0]
cost = costs[0]

In [19]:
cost

{0: {0: 0.0, 1: 0, 2: 160.73, 3: 118.19},
 1: {0: 0.0, 1: 28.67, 2: 60.73, 3: 18.19}}

In [20]:
transition_matrix_[0]

,0.0,1.0,2.0
0,0.000000,0.333333,0.666667
1,0.000000,0.000000,1.000000
2,0.666667,0.000000,0.333333


In [21]:
transition_matrix_[1]

,0.0,1.0,2.0
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.2,0.4,0.4


## Testing the model

In [23]:
agent_dqn = DQNBuild(environment = transition_matrix_, cost = cost, num_states = 3, num_actions = 2,
                    hidden_dim = 64)

In [24]:
agent_dqn.train(episodes = 5, steps_per_episode = 365, batch_size = 32, terminal_period = False, episodic=False, use_target_network=True, target_update_period = 180)

Convergence Reached With Statble Policy at 2
Optimal Policy: [0 1 1]


# Value Iteration Algorithm

In [38]:
def value_iteration(environment, cost, n_actions, tol, n_state):
    # Initialize the value function (V) and policy arrays
    v_new = np.zeros(shape=(len(n_state), 1))

    # initialize the starting policy
    policy_o = np.zeros(shape=v_new.shape)
    # initialize array to store new policy
    policy_n = policy_o.copy()
    
    while True:
        v_n = v_new.copy()
        
        for state in n_state:
            action_value = []
            for action in range(n_actions):
                # Obtain the cost of the action for the current state
                c_i = cost[action][state]
                # Obtain the transition probabilities for the current state and action
                p_ij = environment[action].loc[state].values
                # Calculate the value for the state-action pair
                v_ = (c_i + np.dot(p_ij, v_n)).item()
                # Append the action value
                action_value.append(v_)

            # Update v_new for the current state with the minimum action value
            v_new[state] = np.min(action_value)
            # Update the policy with the action that yields the minimum value
            policy_n[state] = np.argmin(action_value)

           # print('v_new and v_check:{}'.format(v_new - v_n))

        # Calculate convergence bounds
        m_n = np.min(v_new - v_n)  # Lower bound
        M_n = np.max(v_new - v_n)  # Upper bound

        # Convergence check
        if (np.max(np.abs(v_new - v_n)) <= tol) or np.array_equal(policy_n, policy_o):
            break

        # Copy the current values to v_n for this iteration
        policy_o = policy_n
    return policy_n.flatten()

In [39]:
value_iteration(environment = transition_matrix_, cost = cost, n_actions=2, tol = 0.0001, n_state=[0, 1, 2])

array([0., 1., 1.])

Observation:
- DQN model obtains the solution as the Value iteration algorithm. Model works well